In [1]:
import os, argparse
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pytz import timezone
from pathlib import Path
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import wandb

import datasets
# from utils import select_device, natural_keys, gazeto3d, angular, getArch
from utils import select_device, natural_keys, gazeto3d, angular, getArch
from model import L2CS

In [2]:

args = argparse.Namespace()
args.gazeMpiimage_dir = '/project/data/Image'  #real data 
args.gazeMpiilabel_dir = '/project/data/Label'  #real label
# args.output = '/project/results/soutput1/snapshots/'
args.dataset = 'mpiigaze'
args.snapshot='/project/results/soutput2/snapshots/'
# args.evalpath = '/project/results/sroutput1/evaluation/'
args.gpu_id = '0,1,2,3'
args.gpu_id = '0'
args.batch_size = 20
args.arch = 'ResNet50'
args.bins=35
args.angle = 180
args.bin_width = 4


In [3]:
batch_size=args.batch_size
arch=args.arch
data_set=args.dataset
# evalpath =args.evalpath
snapshot_path = args.snapshot
bins=args.bins
angle=args.angle
bin_width=args.bin_width

In [4]:
# args = parse_args()
cudnn.enabled = True
gpu = select_device(args.gpu_id, batch_size=args.batch_size)
transformations = transforms.Compose([
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

model_used=getArch(arch, bins)  #resnet50 and 28 bins

In [5]:
# check if we have the correct number of checkpoint files 
ppath = snapshot_path
for fold in range(15):
    foldstr = f"fold{fold:0>2}"
    cpath =os.path.join(ppath, foldstr)
    files = os.listdir(cpath)
    print(f'{fold}:{len(files)}',end=" ")

0:60 1:60 2:60 3:60 4:60 5:60 6:60 7:60 8:60 9:60 10:60 11:60 12:60 13:60 14:60 

In [6]:
def get_now():
    now = datetime.utcnow()
    now = now.astimezone(timezone('US/Pacific'))
    date_format='%m/%d/%Y %H:%M:%S'
    now = now.strftime(date_format) 
    return now
print(get_now())

07/11/2022 08:50:24


In [ ]:
%%time
all_MAE = []
# tosave={}
for fold in range(15):
    frun = wandb.init(project="61_syn_mode_sdata2_real_data")
    frun.name = f'fold{fold}_mae'
    
    print(f"fold={fold}")
    
    epoch_values=[]
    mae_values=[]

    now = get_now()
    
    print(args.gazeMpiilabel_dir)
    folder = os.listdir(args.gazeMpiilabel_dir)
    folder.sort()  #individual label files
    testlabelpathcombined = [os.path.join(args.gazeMpiilabel_dir, j) for j in folder] 
    
    gaze_dataset=datasets.Mpiigaze(testlabelpathcombined, args.gazeMpiimage_dir, transformations, False, angle, fold)

    test_loader = torch.utils.data.DataLoader(
        dataset=gaze_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4,
        pin_memory=True)

#     fold_path = os.path.join(evalpath, 'fold' + f'{fold:0>2}'+'/')  #for evaluation
#     print(f"fold_path is {fold_path}")
#     if not os.path.exists(fold_path):
#         os.makedirs(fold_path)

#     if not os.path.exists(os.path.join(evalpath, f"fold"+str(fold))):
#         os.makedirs(os.path.join(evalpath, f"fold"+str(fold)))

    # list all epoch for testing
    folder = os.listdir(os.path.join(snapshot_path, "fold" + f'{fold:0>2}'))
    folder.sort(key=natural_keys)
#     folder.pop(-1)  #remove the tensorboard file, now all snapshot files
#     print(f"folder={folder}")
                    

    softmax = nn.Softmax(dim=1)
#     with open(os.path.join(evalpath, os.path.join("fold"+f'{fold:0>2}', data_set+".log")), 'w') as outfile:
        
    configuration = (f"\ntest configuration equal gpu_id={gpu}, batch_size={batch_size}, model_arch={arch}\n"
                     f"Start testing dataset={data_set}, FOLD={fold} --{now}---------")
    print(configuration)
#     tosave['time':now]
#     outfile.write(configuration)
    epoch_list=[]
    avg_MAE=[]
    for epoch in folder: 
        print(f"entering epoch={epoch}")
        model=model_used
        checkpoint = torch.load(os.path.join(snapshot_path+"fold"+f'{fold:0>2}', epoch))
        saved_state_dict = checkpoint['model_state_dict']
        model= nn.DataParallel(model,device_ids=[0])
        model.load_state_dict(saved_state_dict)
        model.cuda(gpu)
        model.eval()
        total = 0
        idx_tensor = [idx for idx in range(35)]
        idx_tensor = torch.FloatTensor(idx_tensor).cuda(gpu)
        avg_error = .0
        with torch.no_grad():
            for j, (images, labels, cont_labels, name) in enumerate(test_loader):
                images = Variable(images).cuda(gpu)
                total += cont_labels.size(0)

                label_pitch = cont_labels[:,0].float()*np.pi/180
                label_yaw = cont_labels[:,1].float()*np.pi/180

                gaze_pitch, gaze_yaw = model(images)

                # Binned predictions
                _, pitch_bpred = torch.max(gaze_pitch.data, 1)
                _, yaw_bpred = torch.max(gaze_yaw.data, 1)

                # Continuous predictions
                pitch_predicted = softmax(gaze_pitch)
                yaw_predicted = softmax(gaze_yaw)

                # mapping from binned (0 to 28) to angels (-42 to 42)                
                pitch_predicted = \
                    torch.sum(pitch_predicted * idx_tensor, 1).cpu() * 3 - 52
                yaw_predicted = \
                    torch.sum(yaw_predicted * idx_tensor, 1).cpu() * 3 - 52

                pitch_predicted = pitch_predicted*np.pi/180
                yaw_predicted = yaw_predicted*np.pi/180

                for p,y,pl,yl in zip(pitch_predicted, yaw_predicted, label_pitch, label_yaw):
                    pl, yl = yl, pl*(-1.0)
#                     yl = yl*(-1.0)
                    avg_error += angular(gazeto3d([p,y]), gazeto3d([pl,yl]))

        x = ''.join(filter(lambda i: i.isdigit(), epoch))
#             print(f"x={x}")
        
        epoch_list.append(x)
        mean_mae = avg_error/total  #mean mae over the 3000 iamges
        avg_MAE.append(mean_mae)  
#             print(f"total={total}")
        now = get_now()
        loger = f"[{epoch}---{args.dataset}] Total Num:{total},MAE:{mean_mae}  {now}"
#         outfile.write(loger)
        print(loger)
#             print(f"done epoch={epochs}")
        epochn = int(x)
#         wandb.log({'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold })
#         tosave ={'epoch':epochn,'total':total, 'MAE':mean_mae, 'FOLD':fold }
        epoch_values.append(epochn)
        mae_values.append(mean_mae)
# #         wandb.log(tosave)
        wandb.log({'epoch': epoch, f'fold_{fold}_avg_mae':mean_mae}, step=epochn)
       
    all_MAE.append(avg_MAE)
    wandb.finish()


wandb: Currently logged in as: synthesis-ai (use `wandb login --relogin` to force relogin)


fold=0
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=0 --07/11/2022 08:50:28---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:6.399443599182071  07/11/2022 08:50:51
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:7.428557377941425  07/11/2022 08:51:06
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:4.518547983982202  07/11/2022 08:51:20
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:4.5789420240625  07/11/2022 08:51:34
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:4.223323457528585  07/11/2022 08:51:49
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:4.246873095102045  07/11/2022 08:52:03
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:4.777595268148093  07/11/2022 08:52:18
enter

fold_0_avg_mae,▆█▂▁▂▂▂▂▁▂▁▁▁▂▁▂▂▂▂▃▃▃▂▂▂▂▁▃▂▂▂▂▃▂▂▂▁▁▂▂
epoch,epoch_60.pkl
fold_0_avg_mae,4.65955


fold=1
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=1 --07/11/2022 09:05:09---------
entering epoch=epoch_1.pkl
[epoch_8.pkl---mpiigaze] Total Num:3000,MAE:6.9937760735096814  07/11/2022 09:07:07
entering epoch=epoch_9.pkl
[epoch_9.pkl---mpiigaze] Total Num:3000,MAE:9.904218630844667  07/11/2022 09:07:21
entering epoch=epoch_10.pkl
[epoch_10.pkl---mpiigaze] Total Num:3000,MAE:8.444881588081254  07/11/2022 09:07:35
entering epoch=epoch_11.pkl
[epoch_11.pkl---mpiigaze] Total Num:3000,MAE:7.10754324667955  07/11/2022 09:07:50
entering epoch=epoch_12.pkl
[epoch_12.pkl---mpiigaze] Total Num:3000,MAE:8.962572431639547  07/11/2022 09:08:04
entering epoch=epoch_13.pkl
[epoch_13.pkl---mpiigaze] Total Num:3000,MAE:5.641800087853064  07/11/2022 09:08:19
entering epoch=epoch_14.pkl
[epoch_14.pkl---mpiigaze] Total Num:3000,MAE:7.62733896000107  07/11/2022 09:

fold_1_avg_mae,▇▇▅█▂▄█▅▁▆▂▄▄▃▃▄▄▂▃▅▄▄▁▄▂▅▃▄▃▄▄▅▆▂▇▃▅▆▂▃
epoch,epoch_60.pkl
fold_1_avg_mae,6.55744


...

fold=12
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=12 --07/11/2022 11:46:37---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:10.106593678996957  07/11/2022 11:46:55
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:10.299983357337705  07/11/2022 11:47:10
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:8.522723181086619  07/11/2022 11:47:24
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:9.76337780829122  07/11/2022 11:47:39
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:8.856436763663167  07/11/2022 11:47:54
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:9.491702402235575  07/11/2022 11:48:08
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:9.282208092068073  07/11/2022 11:48:23


fold_12_avg_mae,▇▇▆▄▅▃▅▄▅▅█▅▃▁▁▅▃▄▁▃▅▂▃▄▅▂▂▅▃▃▆▆▅▄▄▄▄▅▄▅
epoch,epoch_60.pkl
fold_12_avg_mae,9.567


fold=13
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=13 --07/11/2022 12:01:14---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:14.338220346889772  07/11/2022 12:01:31
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:13.018350540535295  07/11/2022 12:01:46
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:11.78913000558656  07/11/2022 12:02:00
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:13.464470480010565  07/11/2022 12:02:14
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:12.558073823655551  07/11/2022 12:02:29
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:9.501403405338639  07/11/2022 12:02:43
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:10.073072561458714  07/11/2022 12:02

fold_13_avg_mae,▇▅▆▅▂▄▆▂▃▃█▄▅▄▃▂▂▂▃▃▃▂▁▃▃▂▃▂▆▃▁▂▃▃▂▆▃▂▃▂
epoch,epoch_60.pkl
fold_13_avg_mae,10.51103


fold=14
/project/data/Label
0 items removed from dataset that have an angle > 180

test configuration equal gpu_id=cuda:0, batch_size=20, model_arch=ResNet50
Start testing dataset=mpiigaze, FOLD=14 --07/11/2022 12:15:51---------
entering epoch=epoch_1.pkl
[epoch_1.pkl---mpiigaze] Total Num:3000,MAE:9.483897073534171  07/11/2022 12:16:08
entering epoch=epoch_2.pkl
[epoch_2.pkl---mpiigaze] Total Num:3000,MAE:8.143008340438856  07/11/2022 12:16:22
entering epoch=epoch_3.pkl
[epoch_3.pkl---mpiigaze] Total Num:3000,MAE:9.298228963167924  07/11/2022 12:16:37
entering epoch=epoch_4.pkl
[epoch_4.pkl---mpiigaze] Total Num:3000,MAE:10.061603253653164  07/11/2022 12:16:52
entering epoch=epoch_5.pkl
[epoch_5.pkl---mpiigaze] Total Num:3000,MAE:8.82462046209274  07/11/2022 12:17:06
entering epoch=epoch_6.pkl
[epoch_6.pkl---mpiigaze] Total Num:3000,MAE:6.966477507628795  07/11/2022 12:17:21
entering epoch=epoch_7.pkl
[epoch_7.pkl---mpiigaze] Total Num:3000,MAE:8.412647805739699  07/11/2022 12:17:35
e

In [ ]:
all_MAE=np.array(all_MAE)

In [ ]:
save_file = Path(snapshot_path).parent/"all_MAE.npy"

In [ ]:
with open(save_file, 'wb') as f:
    np.save(f, all_MAE)

In [ ]:

print(all_MAE.shape)
print(all_MAE.mean(axis=0))
print(all_MAE.mean(axis=1))
print(all_MAE.mean(axis=0).argmin()+1 ,all_MAE.mean(axis=0).min())

In [ ]:
best_epoch = all_MAE.mean(axis=0).argmin()+1
min_error = all_MAE.mean(axis=0).min()
mean_errors = all_MAE.mean(axis=0)
print(best_epoch,min_error, mean_errors.shape )

In [ ]:
xx = mean_errors
min_error = 100
with wandb.init(project="61_syn_mode_sdata2_real_data") as run:
    run.name = "mean_error_per_epoch"
    for epoch in range(len(xx)):
        err = xx[epoch]
        if err < min_error:
            min_error = err
            run.summary["minimum_error (degree)"] = min_error
        run.log({f'mean epoch error (degree)':err}, step=epoch)
        

In [ ]:


best_epoch = {}
best_epoch_error = {}
print(f"fold   best_epoch error(degree)")
for idx, fdata in enumerate(all_MAE):
    epoch_min_error = np.argmin(fdata)+1
    print(f'{idx:^4} {np.argmin(fdata)+1:^12} {fdata.min():^11.3f}')
    best_epoch[idx+1] = epoch_min_error
    best_epoch_error[idx+1]=fdata.min()


In [ ]:
all_MAE = np.load(save_file)

In [ ]:
all_MAE.shape

In [ ]:
plt.plot(all_MAE.min(axis=1))
plt.title('distribution of error over fold')
plt.xlabel('fold')
plt.ylabel('error(degree)')
with wandb.init(project="51_syn_mode_real_data") as run:
    run.log({'data':wandb.Image(plt)})